In [1]:
from google.colab import drive
drive.mount('/content/drive')


import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [2]:
from google.colab import drive
drive.mount('/content/drive')


import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [3]:
# Cell 3: پیش‌پردازش و مهندسی ویژگی
# حذف ستون StudentID (بی‌فایده برای پیش‌بینی)
train = train.drop(columns=['StudentID'])
test = test.drop(columns=['StudentID'])

In [4]:
# Cell 3: پیش‌پردازش و مهندسی ویژگی
# حذف ستون StudentID (بی‌فایده برای پیش‌بینی)
train = train.drop(columns=['StudentID'])
test = test.drop(columns=['StudentID'])

In [5]:
# Cell 2: خواندن مجموعه داده‌ها
train = pd.read_csv('/content/drive/MyDrive/Quera_Data_Mining_2025/Student/data/train.csv')  # داده‌های آموزشی
test = pd.read_csv('/content/drive/MyDrive/Quera_Data_Mining_2025/Student/data/test.csv')    # داده‌های آزمون

In [6]:
# Cell 3: پیش‌پردازش و مهندسی ویژگی
# حذف ستون StudentID (بی‌فایده برای پیش‌بینی)
train = train.drop(columns=['StudentID'])
test = test.drop(columns=['StudentID'])

In [7]:
# مهندسی ویژگی
# ویژگی 1: مجموع فعالیت‌های فوق‌برنامه
train['Total_Activities'] = train['Extracurricular'] + train['Sports'] + train['Music'] + train['Volunteering']
test['Total_Activities'] = test['Extracurricular'] + test['Sports'] + test['Music'] + test['Volunteering']

# ویژگی 2: نسبت زمان مطالعه به غیبت (برای جلوگیری از تقسیم بر صفر)
train['Study_Absence_Ratio'] = train['StudyTimeWeekly'] / (train['Absences'] + 1)
test['Study_Absence_Ratio'] = test['StudyTimeWeekly'] / (test['Absences'] + 1)

# ویژگی 3: تعامل بین StudyTimeWeekly و Tutoring
train['Study_Tutoring_Interaction'] = train['StudyTimeWeekly'] * train['Tutoring']
test['Study_Tutoring_Interaction'] = test['StudyTimeWeekly'] * test['Tutoring']

# شناسایی ویژگی‌ها و هدف
target = 'GPA'
X = train.drop(columns=[target])
y = train[target]

# شناسایی ستون‌های عددی و کته‌گوریکال
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object']).columns

# پیش‌پردازش: مدیریت NaN و انکودینگ
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),  # پر کردن NaN با میانه
            ('scaler', StandardScaler())  # مقیاس‌بندی
        ]), numerical_cols),
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),  # پر کردن NaN با 'missing'
            ('onehot', OneHotEncoder(handle_unknown='ignore', drop='first'))  # انکودینگ کته‌گوریکال
        ]), categorical_cols)
    ]
)

In [8]:
# Cell 4: طراحی مدل
# تعریف مدل RandomForest با GridSearch برای بهینه‌سازی
rf = RandomForestRegressor(random_state=42)
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, scoring='r2')

# ایجاد Pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', grid_search)
])

# تقسیم داده‌ها برای اعتبارسنجی
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# آموزش مدل
model.fit(X_train, y_train)

In [9]:
# Cell 5: ارزیابی مدل
y_pred_val = model.predict(X_val)
r2 = r2_score(y_val, y_pred_val)
print(f'R2 Score on validation: {r2}')
print(f'Best parameters: {model.named_steps["regressor"].best_params_}')

# Cell 6: پیش‌بینی روی داده‌های آزمون و ساخت submission
predictions = model.predict(test)
submission = pd.DataFrame({'GPA': predictions})
submission.to_csv('submission.csv', index=False)